# Lookout Mobile Malware Threat Hunting

This notebook provides threat hunting queries for investigating mobile malware detected by Lookout Mobile Risk API.

## Prerequisites
- Lookout data connector configured in Microsoft Sentinel
- LookoutEvents parser deployed
- Access to Log Analytics workspace

## 1. High Severity Malware Detections

Identify devices with critical or high severity malware threats.

In [ ]:
LookoutEvents
| where EventType == "THREAT"
| where ThreatSeverity in ("CRITICAL", "HIGH")
| where ThreatType in ("MALWARE", "APPLICATION")
| summarize 
    ThreatCount = count(),
    Applications = make_set(ThreatApplicationName),
    PackageHashes = make_set(ThreatPackageSha)
    by DeviceEmailAddress, DevicePlatform, ThreatSeverity
| sort by ThreatCount desc

## 2. Malware Package Hash Analysis

Analyze malicious package hashes across the fleet to identify widespread campaigns.

In [ ]:
LookoutEvents
| where EventType == "THREAT"
| where isnotempty(ThreatPackageSha)
| summarize 
    DevicesAffected = dcount(DeviceEmailAddress),
    FirstSeen = min(TimeGenerated),
    LastSeen = max(TimeGenerated),
    AppNames = make_set(ThreatApplicationName),
    Severities = make_set(ThreatSeverity)
    by ThreatPackageSha
| where DevicesAffected > 1
| sort by DevicesAffected desc

## 3. Malware Timeline Analysis

Track malware detection trends over time to identify attack campaigns.

In [ ]:
LookoutEvents
| where EventType == "THREAT"
| where ThreatType in ("MALWARE", "APPLICATION")
| summarize Count = count() by bin(TimeGenerated, 1h), ThreatSeverity
| render timechart

## 4. Unresolved Malware Threats

Identify devices with malware that hasn't been remediated.

In [ ]:
LookoutEvents
| where EventType == "THREAT"
| where ThreatType in ("MALWARE", "APPLICATION")
| where ThreatStatus == "OPEN"
| project 
    TimeGenerated,
    DeviceEmailAddress,
    DevicePlatform,
    ThreatApplicationName,
    ThreatSeverity,
    ThreatDescription,
    ThreatPackageSha
| sort by TimeGenerated desc